In [62]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [63]:
df = pd.read_csv('../data/process_v1/process_data_6.csv')
df = df[df['city'] == 0]

In [64]:
df = df[df['landSize'] <= 5000]

In [65]:
df['is_street_house'] = (df['accessibility'] == 2).astype(np.int32)

In [66]:
df['landSize_ratio'] = df['landSize'] / df['acreage']

In [67]:
mean_land_size_df = df.groupby('administrative_genre')['landSize'].mean().reset_index().rename(columns = {'landSize': 'meanLandSize'})

df = df.merge(mean_land_size_df, how='left', on = 'administrative_genre')

df['landSize_ratio_with_administrative_genre'] = df['landSize'] / df['meanLandSize']
df['acreage_ratio_with_meanLandSize'] = df['acreage'] / df['meanLandSize']


del df['meanLandSize']

def partition_for_alley_width(alley_width):
    if math.isnan(alley_width):
        return np.nan
    if alley_width <= 2.5:
        return 1
    if alley_width <= 5:
        return 2
    if alley_width <= 7:
        return 3
    if alley_width <= 8:
        return 4

In [68]:
{'fitThreeCars': 0,
 'fitOneCarAndOneMotorbike': 1,
 'notInTheAlley': 2,
 'parkCar': 3,
 'theBottleNeckPoint': 4,
 'fitTwoCars': 5,
 'narrorRoad': 6}

{'fitThreeCars': 0,
 'fitOneCarAndOneMotorbike': 1,
 'notInTheAlley': 2,
 'parkCar': 3,
 'theBottleNeckPoint': 4,
 'fitTwoCars': 5,
 'narrorRoad': 6}

In [69]:
df.columns.tolist()

['numberOfFloors',
 'numberOfBathRooms',
 'numberOfBedRooms',
 'certificateOfLandUseRight',
 'ward',
 'street',
 'district',
 'city',
 'lat',
 'lon',
 'typeOfRealEstate',
 'frontWidth',
 'endWidth',
 'facade',
 'houseDirection',
 'landSize',
 'price',
 'accessibility',
 'description',
 'time',
 'w',
 'h',
 'num_of_university_in_500m_radius',
 'num_of_fuel_in_500m_radius',
 'num_of_cafe_in_500m_radius',
 'num_of_parking_in_500m_radius',
 'num_of_parking_entrance_in_500m_radius',
 'num_of_fast_food_in_500m_radius',
 'num_of_marketplace_in_500m_radius',
 'num_of_restaurant_in_500m_radius',
 'num_of_hospital_in_500m_radius',
 'num_of_school_in_500m_radius',
 'num_of_kindergarten_in_500m_radius',
 'num_of_townhall - community_centre_in_500m_radius',
 'num_of_police_in_500m_radius',
 'num_of_place_of_worship_in_500m_radius',
 'num_of_bank_in_500m_radius',
 'num_of_atm_in_500m_radius',
 'num_of_university_in_1000m_radius',
 'num_of_fuel_in_1000m_radius',
 'num_of_cafe_in_1000m_radius',
 'num_

In [70]:
s1 = u'ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝàáâãèéêìíòóôõùúýĂăĐđĨĩŨũƠơƯưẠạẢảẤấẦầẨẩẪẫẬậẮắẰằẲẳẴẵẶặẸẹẺẻẼẽẾếỀềỂểỄễỆệỈỉỊịỌọỎỏỐốỒồỔổỖỗỘộỚớỜờỞởỠỡỢợỤụỦủỨứỪừỬửỮữỰựỲỳỴỵỶỷỸỹ'
s0 = u'AAAAEEEIIOOOOUUYaaaaeeeiioooouuyAaDdIiUuOoUuAaAaAaAaAaAaAaAaAaAaAaAaEeEeEeEeEeEeEeEeIiIiOoOoOoOoOoOoOoOoOoOoOoOoUuUuUuUuUuUuUuYyYyYyYy'
def remove_accents(input_str):
	s = ''
	for c in input_str:
		if c in s1:
			s += s0[s1.index(c)]
		else:
			s += c

	s = "".join([c for c in list(s) if c == '_' or c.isalnum()])
	return s

full_cols = df.columns.tolist()
format_full_cols = [remove_accents(col) for col in full_cols]

df = df.rename(columns = dict(zip(full_cols, format_full_cols)))

In [71]:
cat_cols = ['is_street_house', 'ward', 'street', 'district', 'typeOfRealEstate', 'houseDirection', 'accessibility', 'certificateOfLandUseRight'] + [c for c in df.columns.tolist() if ('_district' in c or '_ward' in c or '_street' in c) and 'distance' not in c and 'num' not in c]
target_cols = ['target']
remove_cols = ['price', 'description', 'city', 'endWidth'] + [c for c in df.columns if 'distance_hcm' in c]
time_cols = ['time']
num_cols = [c for c in df.columns.tolist() if c not in (cat_cols + target_cols + remove_cols + time_cols)]

cat_cols = list(set(cat_cols))
target_cols = list(set(target_cols))
remove_cols = list(set(remove_cols))
num_cols = list(set(num_cols))

print(len(cat_cols), len(target_cols), len(remove_cols), len(num_cols))

35 1 53 148


In [72]:
cat_cols

['nearest_0_ward',
 'nearest_8_ward',
 'certificateOfLandUseRight',
 'nearest_4_district',
 'nearest_7_district',
 'nearest_3_district',
 'nearest_1_district',
 'nearest_6_street',
 'nearest_5_district',
 'nearest_2_ward',
 'houseDirection',
 'nearest_1_ward',
 'nearest_4_ward',
 'nearest_5_ward',
 'nearest_6_ward',
 'typeOfRealEstate',
 'nearest_7_street',
 'district',
 'street',
 'nearest_3_ward',
 'is_street_house',
 'nearest_3_street',
 'nearest_2_street',
 'nearest_1_street',
 'nearest_8_district',
 'nearest_8_street',
 'nearest_7_ward',
 'nearest_0_street',
 'nearest_6_district',
 'ward',
 'nearest_2_district',
 'nearest_5_street',
 'nearest_0_district',
 'nearest_4_street',
 'accessibility']

In [73]:
num_cols

['num_of_fuel_in_1000m_radius',
 'nearest_3_lat',
 'distance_hn_NguyenTriPhuong',
 'distance_nearest_5',
 'distance_hn_DoanVanBo',
 'distance_nearest_1',
 'w',
 'num_of_townhallcommunity_centre_in_500m_radius',
 'num_of_atm_in_1000m_radius',
 'nearest_8_lon',
 'distance_hn_CongviennuocCuChi',
 'distance_hn_AeonMallShoppingCenterTanPhu',
 'num_of_cafe_in_2000m_radius',
 'num_of_parking_in_500m_radius',
 'num_of_marketplace_in_1000m_radius',
 'num_of_bank_in_2000m_radius',
 'distance_hn_DuongNguyenHuequan1TPHCM',
 'distance_hn_LeVanSy',
 'num_of_kindergarten_in_2000m_radius',
 'distance_hn_CongtruongLamSon',
 'distance_hn_CongviencauAnhSaoHoBanNguyet',
 'nearest_5_lon',
 'distance_nearest_0',
 'num_of_kindergarten_in_500m_radius',
 'district_lat',
 'distance_hn_KinhDuongVuong',
 'distance_hn_BaHuyenThanhQuan',
 'nearest_4_lon',
 'distance_hn_CrescentMall',
 'distance_hn_DuongNamKyKhoiNghia',
 'distance_hn_CongvienTaoDan',
 'distance_hn_Congvien304',
 'distance_hn_CongvienkhudothiSala',
 

In [74]:
print(len(num_cols))

148


In [75]:
df.columns.tolist()

['numberOfFloors',
 'numberOfBathRooms',
 'numberOfBedRooms',
 'certificateOfLandUseRight',
 'ward',
 'street',
 'district',
 'city',
 'lat',
 'lon',
 'typeOfRealEstate',
 'frontWidth',
 'endWidth',
 'facade',
 'houseDirection',
 'landSize',
 'price',
 'accessibility',
 'description',
 'time',
 'w',
 'h',
 'num_of_university_in_500m_radius',
 'num_of_fuel_in_500m_radius',
 'num_of_cafe_in_500m_radius',
 'num_of_parking_in_500m_radius',
 'num_of_parking_entrance_in_500m_radius',
 'num_of_fast_food_in_500m_radius',
 'num_of_marketplace_in_500m_radius',
 'num_of_restaurant_in_500m_radius',
 'num_of_hospital_in_500m_radius',
 'num_of_school_in_500m_radius',
 'num_of_kindergarten_in_500m_radius',
 'num_of_townhallcommunity_centre_in_500m_radius',
 'num_of_police_in_500m_radius',
 'num_of_place_of_worship_in_500m_radius',
 'num_of_bank_in_500m_radius',
 'num_of_atm_in_500m_radius',
 'num_of_university_in_1000m_radius',
 'num_of_fuel_in_1000m_radius',
 'num_of_cafe_in_1000m_radius',
 'num_of_

In [76]:
df[cat_cols] = df[cat_cols].fillna(100)
df[cat_cols] = df[cat_cols].astype(np.int32)

In [77]:
df[num_cols]

,num_of_fuel_in_1000m_radius,nearest_3_lat,distance_hn_NguyenTriPhuong,distance_nearest_5,distance_hn_DoanVanBo,distance_nearest_1,w,num_of_townhallcommunity_centre_in_500m_radius,num_of_atm_in_1000m_radius,nearest_8_lon,...,nearest_5_lat,acreage,population,num_of_restaurant_in_2000m_radius,num_of_school_in_2000m_radius,num_of_police_in_500m_radius,num_of_marketplace_in_2000m_radius,num_of_cafe_in_500m_radius,num_of_marketplace_in_500m_radius,distance_hn_LeHongPhong
0,1,10.855713,13.942570,NaN,14.088371,6.345885,0.0,0,0,NaN,...,NaN,3.871201,13.170525,4,19,0,6,0,0,9.509579
1,0,10.723160,13.955746,6.399320,14.091541,5.587561,0.0,0,8,106.708597,...,10.727941,3.574871,12.690411,71,26,0,1,8,0,8.749448
2,6,10.742963,13.953660,5.832474,14.090865,5.480425,4.0,0,7,106.715936,...,10.741552,3.574871,12.690411,64,31,0,6,7,1,8.564476
3,6,10.799577,13.947782,5.535487,14.083803,5.118292,0.0,0,10,106.652023,...,10.797977,3.108168,13.061232,45,54,0,8,6,1,8.529008
4,5,10.790682,13.948927,5.479074,14.087347,4.875554,4.0,0,26,106.689549,...,10.793641,2.045109,12.231643,370,66,0,8,19,2,8.352073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48888,0,10.867586,13.941188,NaN,14.076348,6.798738,14.0,0,0,NaN,...,NaN,4.691348,13.000130,2,7,0,1,0,0,9.620117
48889,4,10.866162,13.940738,6.359943,14.078927,6.151324,4.0,0,0,106.624451,...,10.865114,3.966132,13.161921,7,10,0,4,0,2,9.511700
48890,1,10.850430,13.943656,7.033275,14.092472,6.564167,5.0,0,2,106.780098,...,10.848256,4.736198,12.891692,9,17,0,2,1,0,9.727426
48891,15,10.752622,13.952693,5.691781,14.087645,4.716857,0.0,0,9,106.679095,...,10.754168,1.451614,14.442256,118,83,0,26,2,1,7.022286


In [78]:
df = df[cat_cols + num_cols + time_cols + target_cols]

In [79]:
df = df[df[target_cols[0]] != np.inf]
df = df[df[target_cols[0]] != -np.inf]

In [80]:
df.shape

(48893, 185)

In [81]:
df = df[df[target_cols[0]] >= 10]
df = df[df[target_cols[0]] < 100 * 10]
# df = pd.concat([df[df['w'] < 50], df[df['w'].isnull()]])
# df = pd.concat([df[df['h'] < 50], df[df['w'].isnull()]])

In [82]:
for col in num_cols:
    df[col] = df[col].fillna(df[col].mean())

In [83]:
for c in df.columns.tolist():
    if df[c].isnull().sum():
        print(c)
        del df[c]

In [84]:
gmm_config = {
    'num_of_restaurant_in_2000m_radius': 2,
    'num_of_restaurant_in_1000m_radius': 2,
    'num_of_bank_in_2000m_radius': 3,
    'num_of_marketplace_in_2000m_radius': 2,
    'num_of_cafe_in_1000m_radius': 2
}

In [85]:
from sklearn.mixture import GaussianMixture

In [86]:
for col in gmm_config.keys():
    gmm = GaussianMixture(n_components=gmm_config[col], random_state=42)
    gmm.fit(df[col].values.reshape(-1, 1))
    df[f'gmm_{gmm_config[col]}_component_{col}'] = gmm.predict(df[col].values.reshape(-1, 1))

/tmp/ipykernel_3140174/1335931363.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'gmm_{gmm_config[col]}_component_{col}'] = gmm.predict(df[col].values.reshape(-1, 1))
/tmp/ipykernel_3140174/1335931363.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'gmm_{gmm_config[col]}_component_{col}'] = gmm.predict(df[col].values.reshape(-1, 1))
/tmp/ipykernel_3140174/1335931363.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performan

,nearest_0_ward,nearest_8_ward,certificateOfLandUseRight,nearest_4_district,nearest_7_district,nearest_3_district,nearest_1_district,nearest_6_street,nearest_5_district,nearest_2_ward,...,num_of_cafe_in_500m_radius,num_of_marketplace_in_500m_radius,distance_hn_LeHongPhong,time,target,gmm_2_component_num_of_restaurant_in_2000m_radius,gmm_2_component_num_of_restaurant_in_1000m_radius,gmm_3_component_num_of_bank_in_2000m_radius,gmm_2_component_num_of_marketplace_in_2000m_radius,gmm_2_component_num_of_cafe_in_1000m_radius
0,0,100,0,100,100,0,0,100,100,541,...,0,0,9.509579,2023-12-27T00:00:00,25.862069,0,0,1,1,0
1,1,1,1,1,1,1,1,2155,1,1,...,8,0,8.749448,2023-12-25T00:00:00,56.666667,0,1,1,1,1
2,5,5,1,1,1,1,1,3374,1,203,...,7,1,8.564476,2023-12-30T00:00:00,184.615385,0,0,1,1,0
3,11,11,1,10,10,10,10,3382,10,11,...,6,1,8.529008,2023-11-27T00:00:00,168.235294,0,0,0,1,0
4,13,13,1,11,11,11,11,3249,11,13,...,19,2,8.352073,2024-01-03T00:00:00,53.846154,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48888,266,100,1,35,100,35,35,100,100,461,...,0,0,9.620117,2023-12-13T00:00:00,21.311475,0,0,1,1,0
48889,347,347,1,13,35,13,35,133,13,389,...,0,2,9.511700,2023-12-14T00:00:00,65.384615,0,0,1,1,0
48890,89,152,1,15,15,15,0,4010,15,152,...,1,0,9.727426,2023-12-14T00:00:00,81.395349,0,0,1,1,0
48891,61,280,1,26,26,26,26,2084,26,280,...,2,1,7.022286,2023-12-01T00:00:00,281.250000,1,1,0,0,0


In [89]:
cat_cols = ['is_street_house', 'ward', 'street', 'district', 'typeOfRealEstate', 'houseDirection', 'accessibility', 'certificateOfLandUseRight'] + [c for c in df.columns.tolist() if ('_district' in c or '_ward' in c or '_street' in c) and 'distance' not in c and 'num' not in c]
cat_cols += [c for c in df.columns if 'gmm' in c]
cat_cols = list(set(cat_cols))
target_cols = ['target']
remove_cols = ['price', 'description', 'city', 'endWidth'] + [c for c in df.columns if 'distance_hcm' in c]
time_cols = ['time']
num_cols = [c for c in df.columns.tolist() if c not in (cat_cols + target_cols + remove_cols + time_cols)]

cat_cols = list(set(cat_cols))
target_cols = list(set(target_cols))
remove_cols = list(set(remove_cols))
num_cols = list(set(num_cols))

print(len(cat_cols), len(target_cols), len(remove_cols), len(num_cols))

40 1 4 148


In [90]:
cat_cols

['nearest_0_ward',
 'nearest_8_ward',
 'certificateOfLandUseRight',
 'nearest_4_district',
 'nearest_7_district',
 'gmm_2_component_num_of_marketplace_in_2000m_radius',
 'nearest_3_district',
 'nearest_1_district',
 'nearest_6_street',
 'nearest_5_district',
 'nearest_2_ward',
 'gmm_2_component_num_of_cafe_in_1000m_radius',
 'houseDirection',
 'gmm_3_component_num_of_bank_in_2000m_radius',
 'nearest_1_ward',
 'nearest_4_ward',
 'nearest_5_ward',
 'nearest_6_ward',
 'typeOfRealEstate',
 'nearest_7_street',
 'district',
 'gmm_2_component_num_of_restaurant_in_1000m_radius',
 'street',
 'nearest_3_ward',
 'is_street_house',
 'nearest_3_street',
 'nearest_2_street',
 'nearest_1_street',
 'nearest_8_district',
 'nearest_8_street',
 'nearest_7_ward',
 'nearest_0_street',
 'nearest_6_district',
 'ward',
 'gmm_2_component_num_of_restaurant_in_2000m_radius',
 'nearest_2_district',
 'nearest_5_street',
 'nearest_0_district',
 'nearest_4_street',
 'accessibility']

In [91]:
num_cols

['num_of_fuel_in_1000m_radius',
 'nearest_3_lat',
 'distance_hn_NguyenTriPhuong',
 'distance_nearest_5',
 'distance_hn_DoanVanBo',
 'distance_nearest_1',
 'w',
 'num_of_townhallcommunity_centre_in_500m_radius',
 'num_of_atm_in_1000m_radius',
 'nearest_8_lon',
 'distance_hn_CongviennuocCuChi',
 'distance_hn_AeonMallShoppingCenterTanPhu',
 'num_of_cafe_in_2000m_radius',
 'num_of_parking_in_500m_radius',
 'num_of_marketplace_in_1000m_radius',
 'num_of_bank_in_2000m_radius',
 'distance_hn_LeVanSy',
 'distance_hn_DuongNguyenHuequan1TPHCM',
 'num_of_kindergarten_in_2000m_radius',
 'distance_hn_CongtruongLamSon',
 'distance_hn_CongviencauAnhSaoHoBanNguyet',
 'nearest_5_lon',
 'distance_nearest_0',
 'num_of_kindergarten_in_500m_radius',
 'district_lat',
 'distance_hn_KinhDuongVuong',
 'distance_hn_BaHuyenThanhQuan',
 'nearest_4_lon',
 'distance_hn_CrescentMall',
 'distance_hn_DuongNamKyKhoiNghia',
 'distance_hn_CongvienTaoDan',
 'distance_hn_Congvien304',
 'distance_hn_CongvienkhudothiSala',
 

In [92]:
df = df.reset_index(drop = True)
df.to_csv('../data/process_v1/process_data_8_hcm.csv', index = False)